In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

print("TensorFlow:", tf.__version__)


DATA_DIR = r"E:\Download\CVPRStudentDataset"   
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

EPOCHS_HEAD = 10
EPOCHS_FINE = 10

MODEL_OUT = "face_resnet_model.h5"
CLASSES_OUT = "class_names.txt"

print("Dataset exists:", os.path.isdir(DATA_DIR))


TensorFlow: 2.20.0
Dataset exists: True


In [2]:
from PIL import Image

ALLOWED_EXT = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

def clean_corrupted_images(data_dir: str):
    removed = 0
    checked = 0
    for root, _, files in os.walk(data_dir):
        for fn in files:
            if not fn.lower().endswith(ALLOWED_EXT):
                continue
            path = os.path.join(root, fn)
            checked += 1
            try:
                with Image.open(path) as im:
                    im.verify()
            except Exception:
                print("Removing corrupted image:", path)
                try:
                    os.remove(path)
                    removed += 1
                except Exception as e:
                    print("   Could not remove:", e)
    print(f"Checked: {checked} | Removed: {removed}")

def remove_junk_files(data_dir: str):
    junk = {"Thumbs.db", ".DS_Store"}
    removed = 0
    for root, _, files in os.walk(data_dir):
        for fn in files:
            if fn in junk:
                path = os.path.join(root, fn)
                try:
                    os.remove(path)
                    removed += 1
                    print("🗑️ Removed junk:", path)
                except Exception as e:
                    print("   Could not remove:", e)
    print("Removed junk files:", removed)


remove_junk_files(DATA_DIR)
clean_corrupted_images(DATA_DIR)


Removed junk files: 0
Checked: 659 | Removed: 0


In [4]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.08,
    height_shift_range=0.08,
    zoom_range=0.10,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

num_classes = train_gen.num_classes
print("num_classes:", num_classes)

# Safety: ensure generator filepaths exist (fixes FileNotFoundError during fit)
missing = [p for p in (train_gen.filepaths + val_gen.filepaths) if not os.path.exists(p)]
print("Missing files:", len(missing))
if missing:
    print("Example missing path:", missing[0])
    raise FileNotFoundError("Some generator filepaths are missing. Rebuild dataset / re-run generators.")


Found 532 images belonging to 39 classes.
Found 126 images belonging to 39 classes.
num_classes: 39
Missing files: 0


In [5]:
idx_to_class = {v: k for k, v in train_gen.class_indices.items()}
class_names = [idx_to_class[i] for i in range(len(idx_to_class))]

with open(CLASSES_OUT, "w", encoding="utf-8") as f:
    for c in class_names:
        f.write(c + "\n")

print("Saved:", CLASSES_OUT)
print("First classes:", class_names[:10])


Saved: class_names.txt
First classes: ['21-45902-3', '22-40000-3', '22-46138-1', '22-46141-1', '22-46156-1', '22-46258-1', '22-46275-1', '22-46473-1', '22-46590-1', '22-46679-1']


In [7]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Stage 1: Freeze backbone
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 39)             │        10,023 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,122,279 (92.02 MB)

 Trainable params: 534,567 (2.04 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(MODEL_OUT, save_best_only=True),
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_HEAD,
    callbacks=callbacks
)

print("✅ Stage 1 complete. Best model saved to:", MODEL_OUT)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.1251 - loss: 3.7373       

17/17 ━━━━━━━━━━━━━━━━━━━━ 157s 8s/step - accuracy: 0.2312 - loss: 3.0511 - val_accuracy: 0.7063 - val_loss: 1.5588
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5760 - loss: 1.6614  

17/17 ━━━━━━━━━━━━━━━━━━━━ 133s 8s/step - accuracy: 0.6485 - loss: 1.4168 - val_accuracy: 0.8413 - val_loss: 0.7068
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7834 - loss: 0.7982  

17/17 ━━━━━━━━━━━━━━━━━━━━ 199s 12s/step - accuracy: 0.7650 - loss: 0.8403 - val_accuracy: 0.9365 - val_loss: 0.4059
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8501 - loss: 0.5693  

17/17 ━━━━━━━━━━━━━━━━━━━━ 105s 6s/step - accuracy: 0.8647 - loss: 0.5488 - val_accuracy: 0.9603 - val_loss: 0.2869
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9219 - loss: 0.3651  

17/17 ━━━━━━━━━━━━━━━━━━━━ 63s 4s/step - accuracy: 0.9154 - loss: 0.3549 - val_accuracy: 0.9921 - val_loss: 0.1657
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9417 - loss: 0.2691 - val_accuracy: 0.9444 - val_loss: 0.2402
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9437 - loss: 0.2513 

17/17 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.9417 - loss: 0.2463 - val_accuracy: 0.9762 - val_loss: 0.1346
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9738 - loss: 0.1642 

17/17 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.9662 - loss: 0.1628 - val_accuracy: 0.9841 - val_loss: 0.1226
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9620 - loss: 0.1629 

17/17 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 0.9718 - loss: 0.1242 - val_accuracy: 0.9841 - val_loss: 0.0907
Epoch 10/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.9850 - loss: 0.0993 - val_accuracy: 0.9841 - val_loss: 0.1102
✅ Stage 1 complete. Best model saved to: face_resnet_model.h5


In [9]:

train_gen.reset()
val_gen.reset()

# Stage 2: Fine-tune
base_model.trainable = True

# Freeze early layers; train last 50 layers (adjust if needed)
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

fine_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_FINE,
    callbacks=callbacks
)

print(" Fine-tuning complete. Best model saved to:", MODEL_OUT)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.9305 - loss: 0.3024 - val_accuracy: 0.9603 - val_loss: 0.1668
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - accuracy: 0.9887 - loss: 0.0721 - val_accuracy: 0.9683 - val_loss: 0.1216
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9944 - loss: 0.0390 - val_accuracy: 0.9683 - val_loss: 0.1080
✅ Fine-tuning complete. Best model saved to: face_resnet_model.h5


In [10]:
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"Validation accuracy: {val_acc*100:.2f}%")


Validation accuracy: 93.65%


In [11]:
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input


In [12]:

MODEL_PATH = "face_resnet_model.h5"
CLASSES_TXT = "class_names.txt"
ATT_CSV = "attendance.csv"      # saves in notebook folder

IMG_SIZE = (224, 224)
CONF_THRESH = 0.70              # adjust: 0.60 ~ 0.85 depending on your model

print("Model exists:", os.path.exists(MODEL_PATH))
print("Classes exists:", os.path.exists(CLASSES_TXT))
print("Attendance path:", os.path.abspath(ATT_CSV))


Model exists: True
Classes exists: True
Attendance path: C:\Users\DELL\attendance.csv


In [13]:
model = load_model(MODEL_PATH)

with open(CLASSES_TXT, "r", encoding="utf-8") as f:
    class_names = [line.strip() for line in f if line.strip()]

print("Loaded model ")
print("Num classes:", len(class_names))
print("First classes:", class_names[:5])


Loaded model ✅
Num classes: 39
First classes: ['21-45902-3', '22-40000-3', '22-46138-1', '22-46141-1', '22-46156-1']


In [14]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def preprocess_face(face_bgr):
    face = cv2.resize(face_bgr, IMG_SIZE)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    face = preprocess_input(face)              # ResNet preprocess
    return np.expand_dims(face, axis=0)


In [15]:
def mark_attendance(student_id: str) -> bool:
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M:%S")

    try:
        df = pd.read_csv(ATT_CSV)
    except FileNotFoundError:
        df = pd.DataFrame(columns=["student_id", "date", "time"])

    already = ((df["student_id"] == student_id) & (df["date"] == date_str)).any()
    if already:
        return False

    df = pd.concat([df, pd.DataFrame([{
        "student_id": student_id,
        "date": date_str,
        "time": time_str
    }])], ignore_index=True)

    df.to_csv(ATT_CSV, index=False)
    return True


In [16]:
cap = cv2.VideoCapture(0)

last_label = "No face"
last_conf = 0.0

print("Controls: M=mark | Q=quit")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Camera read failed.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    if len(faces) > 0:
        (x, y, w, h) = max(faces, key=lambda b: b[2]*b[3])
        face = frame[y:y+h, x:x+w]

        probs = model.predict(preprocess_face(face), verbose=0)[0]
        idx = int(np.argmax(probs))
        conf = float(probs[idx])

        if conf >= CONF_THRESH and idx < len(class_names):
            last_label = class_names[idx]
            last_conf = conf
        else:
            last_label = "Unknown"
            last_conf = conf

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    else:
        last_label = "No face"
        last_conf = 0.0

    cv2.putText(frame, f"{last_label} ({last_conf:.2f})",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                (255, 255, 255), 2)

    cv2.imshow("Attendance (ResNet50)", frame)

    k = cv2.waitKey(1) & 0xFF
    if k in [ord("q"), ord("Q")]:
        break

    if k in [ord("m"), ord("M")]:
        if last_label not in ["Unknown", "No face"]:
            ok = mark_attendance(last_label)
            print(" Marked!" if ok else " Already marked today.")
        else:
            print(" Unknown / No face -> not marked.")

cap.release()
cv2.destroyAllWindows()

print("Saved attendance to:", os.path.abspath(ATT_CSV))


Controls: M=mark | Q=quit
⚠️ Already marked today.
Saved attendance to: C:\Users\DELL\attendance.csv


In [18]:
if os.path.exists(ATT_CSV):
    df = pd.read_csv(ATT_CSV).sort_values(["date", "time"], ascending=False)
    df
else:
    print("No attendance.csv found yet.")
